# ALeRCE classes

https://github.com/ZwickyTransientFacility/ztf-avro-alert

1. **AGN:** Active Galactic Nuclei
1. **Blazar:** Blazar
1. **CV/Nova:** Cataclysmic Variable Star/Nova
1. **Ceph: Cepheid Variable Star**
1. **DSCT: Delta Scuti Star**
1. **EA:** Eclipsing Algol
1. **EB/EW: Eclipsing Binaries/Eclipsing W Ursa Majoris**
1. **LPV: Long Period Variable**
1. **Periodic-Other:** Periodic-Other
1. **QSO:** Quasi-Stellar Object
1. **RRL: RRLyrae Variable Star**
1. **RSCVn:** RS Canum Venaticorum
1. **SLSN:** Super Luminous Supernova
1. **SNII:** Supernova II
1. **SNIIb:** Supernova IIb
1. **SNIIn:** Supernova IIn
1. **SNIa:** Supernova Ia
1. **SNIbc:** Supernova Ibc
1. **TDE:** Tidal disruption event (to remove)
1. **YSO:** Young Stellar Object
1. **ZZ:** ZZ Ceti Stars (to remove)

# Columns names
1. **oid:** object id
1. **classALeRCE:** object class name
1. **fid:** band index, g=1, r=2
1. **mjd:** observation day in mjd format
1. **magpsf_corr:** observations in aparent magnitude
1. **sigmapsf_corr:** observation errors in aparent magnitude

In [ ]:
import sys
sys.path.append('../')
sys.path.append('../../')

In [ ]:
import numpy as np
import pandas as pd
            
#survey_name = 'alerceZTFv5.1'
survey_name = 'alerceZTFv7.1' # use this dataset

### load files
load_root_dir = f'../data/{survey_name}'
labels_df = pd.read_parquet(f'{load_root_dir}/labels_vs.parquet')
print(f'labels_df; columns={list(labels_df.columns)}; id={labels_df.index.name}')
detections_df = pd.read_parquet(f'{load_root_dir}/detections_vs.parquet')
print(f'detections_df; columns={list(detections_df.columns)}; id={detections_df.index.name}')

### print info
classes = sorted(set(labels_df['classALeRCE'].values))
print(f'classes={classes}')

In [ ]:
%load_ext autoreload
%autoreload 2
from fuzzytools.level_bars import LevelBar

classes, counts = np.unique(labels_df['classALeRCE'].values, return_counts=True)
population_cdict = {c:counts[kc] for kc,c in enumerate(classes)}
print(LevelBar(population_cdict, ncols=60))

In [ ]:
print(detections_df.info())
detections_df[:20]

In [ ]:
print(labels_df.info())
labels_df[:20]

In [ ]:
%load_ext autoreload
%autoreload 2
from vstars import _C
from vstars.utils import get_object
import matplotlib.pyplot as plt
from dask import dataframe as dd

### example using dask
detections_ddf = dd.from_pandas(detections_df, npartitions=_C.N_DASK) # dask dataframe can be faster
labels_ddf = dd.from_pandas(labels_df, npartitions=_C.N_DASK) # dask dataframe can be faster
obj_name = 'ZTF18abvpirg'
band = 1
days, obs, obs_error, c = get_object(detections_ddf, labels_ddf, obj_name, band)

color = 'k'
fig, ax = plt.subplots(1, 1, figsize=(10,4))
ax.plot(days, obs, 'o--', c=color)
ax.errorbar(days, obs, yerr=obs_error, color=color, capsize=0, elinewidth=1, linewidth=0)
ax.set_title(f'obj={obj_name}; class={c}; band={band}')
ax.grid(alpha=0.5)
plt.show()
print(labels_df.loc[obj_name])
print(detections_df.loc[obj_name])